# NBA Player Performance Dynamics: Multi-Dimensional Player Impact

This notebook builds on the network analysis from the previous notebook to develop a comprehensive player impact framework that integrates production, stability, adaptability, and network metrics.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from datetime import datetime
import networkx as nx
from sklearn.preprocessing import StandardScaler

# Add the project root to the path so we can import our modules
sys.path.append('..')

# Import our modules
from src.network_analysis import calculate_network_value
from src.visualization import create_player_impact_cards
from src.utils import setup_plotting_style

# Set up plotting style
setup_plotting_style()

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## Load Processed Data

Let's load the processed data from previous notebooks, including the network metrics we calculated in the previous notebook.

In [ ]:
# Load the processed data
try:
    player_dynamics = pd.read_csv('../data/processed/player_dynamics.csv')
    player_team_fit = pd.read_csv('../data/processed/player_team_fit.csv')
    network_metrics = pd.read_csv('../data/processed/network_metrics.csv')
    synergy_pairs = pd.read_csv('../data/processed/synergy_pairs.csv')
    player_temporal_df = pd.read_csv('../data/processed/player_temporal.csv')
    games_processed = pd.read_csv('../data/processed/games_processed.csv')
    
    # Convert date strings to datetime objects
    player_temporal_df['GAME_DATE'] = pd.to_datetime(player_temporal_df['GAME_DATE'])
    games_processed['GAME_DATE'] = pd.to_datetime(games_processed['GAME_DATE'])
    
    print(f"Loaded player dynamics data with {len(player_dynamics)} players")
    print(f"Loaded player team fit data with {len(player_team_fit)} players")
    print(f"Loaded network metrics data with {len(network_metrics)} players")
    print(f"Loaded synergy pairs data with {len(synergy_pairs)} pairs")
    print(f"Loaded player temporal data with {len(player_temporal_df)} records")
    print(f"Loaded processed games data with {len(games_processed)} records")
except FileNotFoundError:
    print("Processed data not found. Please run the previous notebooks first.")

In [ ]:
# Examine the network metrics data
network_metrics.head()

## Multi-Dimensional Player Impact

Let's integrate production, stability, adaptability, and network metrics to create a comprehensive player impact framework.

In [ ]:
# Merge all player metrics
# Start with player dynamics
player_impact = player_dynamics.copy()

# Add team fit metrics
player_impact = pd.merge(
    player_impact,
    player_team_fit[['player_id', 'adaptability_score', 'best_style']],
    on='player_id',
    how='left'
)

# Add network metrics
player_impact = pd.merge(
    player_impact,
    network_metrics,
    on=['player_id', 'player_name'],
    how='left'
)

# Fill missing values
player_impact = player_impact.fillna(0)

# Display the merged data
player_impact.head()

In [ ]:
# Calculate comprehensive impact score
# Normalize metrics for fair comparison
scaler = StandardScaler()

# Select metrics for impact score
impact_metrics = ['avg_pts', 'avg_plus_minus', 'system_stability', 'adaptability_score', 'eigenvector_centrality', 'avg_influence']

# Create a copy of the data for scaling
impact_data = player_impact[impact_metrics].copy()

# Invert system_stability (lower is better)
impact_data['system_stability'] = -impact_data['system_stability']

# Scale the data
impact_scaled = scaler.fit_transform(impact_data)

# Create a dataframe with scaled metrics
impact_scaled_df = pd.DataFrame(impact_scaled, columns=impact_metrics)

# Calculate impact score with weights
weights = {
    'avg_pts': 0.2,
    'avg_plus_minus': 0.3,
    'system_stability': 0.15,
    'adaptability_score': 0.15,
    'eigenvector_centrality': 0.1,
    'avg_influence': 0.1
}

impact_score = np.zeros(len(impact_scaled_df))
for metric, weight in weights.items():
    impact_score += weight * impact_scaled_df[metric]

# Add impact score to player impact dataframe
player_impact['impact_score'] = impact_score

# Sort by impact score
player_impact = player_impact.sort_values('impact_score', ascending=False)

# Display top players by impact score
print("Top Players by Comprehensive Impact Score:")
player_impact[['player_name', 'impact_score', 'avg_pts', 'avg_plus_minus', 'system_stability', 'adaptability_score', 'eigenvector_centrality', 'avg_influence']].head(10)

In [ ]:
# Visualize impact score components for top players
top_players = player_impact.head(10)

# Create a radar chart for each player
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D

# Radar chart function
def radar_chart(ax, angles, values, color, label):
    # Plot data
    ax.plot(angles, values, 'o-', linewidth=2, color=color, label=label)
    # Fill area
    ax.fill(angles, values, alpha=0.25, color=color)
    # Set y-ticks
    ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
    # Set category labels
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories)
    # Add legend
    ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))

# Categories for the radar chart
categories = ['Scoring', 'Impact', 'Stability', 'Adaptability', 'Centrality', 'Influence']
# Number of categories
N = len(categories)
# Angle of each axis
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]  # Close the loop

# Create figure
fig, axes = plt.subplots(2, 3, figsize=(18, 12), subplot_kw=dict(polar=True))
axes = axes.flatten()

# Colors for each player
colors = plt.cm.tab10(np.linspace(0, 1, len(top_players)))

# Normalize values for radar chart
max_values = {
    'avg_pts': top_players['avg_pts'].max(),
    'avg_plus_minus': top_players['avg_plus_minus'].max(),
    'system_stability': -top_players['system_stability'].min(),  # Invert for radar chart
    'adaptability_score': top_players['adaptability_score'].max(),
    'eigenvector_centrality': top_players['eigenvector_centrality'].max(),
    'avg_influence': top_players['avg_influence'].max()
}

# Plot each player
for i, (_, player) in enumerate(top_players.iterrows()):
    if i < 6:  # Only plot the top 6 players
        # Get normalized values
        values = [
            player['avg_pts'] / max_values['avg_pts'],
            player['avg_plus_minus'] / max_values['avg_plus_minus'],
            -player['system_stability'] / max_values['system_stability'],  # Invert for radar chart
            player['adaptability_score'] / max_values['adaptability_score'],
            player['eigenvector_centrality'] / max_values['eigenvector_centrality'],
            player['avg_influence'] / max_values['avg_influence']
        ]
        values += values[:1]  # Close the loop
        
        # Plot radar chart
        radar_chart(axes[i], angles, values, colors[i], player['player_name'])
        axes[i].set_title(f"{player['player_name']}\nImpact Score: {player['impact_score']:.2f}")

plt.tight_layout()
plt.show()

### Interpreting Multi-Dimensional Impact

Our comprehensive impact framework integrates multiple dimensions of player performance to provide a more complete picture of player value:

1. **Top Impact Players**: The players with the highest impact scores excel across multiple dimensions, combining production, stability, adaptability, and network influence.

2. **Impact Profiles**: Different players derive their value from different combinations of dimensions:
   - Some players derive their value primarily from scoring and traditional production
   - Others contribute through stability, adaptability, or network effects
   - The most valuable players excel in multiple dimensions

3. **Balanced Impact**: Players with balanced impact across all dimensions tend to contribute to team success in multiple ways, making them particularly valuable for team construction.

This multi-dimensional approach provides a more nuanced understanding of player value than traditional statistics alone, capturing aspects of performance that are invisible to conventional metrics.

## Traditional vs. Dynamics Comparison

Let's compare our dynamics-based player evaluation with traditional methods to identify undervalued and overvalued players.

In [ ]:
# Calculate traditional player rating
player_impact['traditional_rating'] = player_impact['avg_pts'] * 0.7 + player_impact['avg_plus_minus'] * 0.3

# Calculate rating difference
player_impact['rating_difference'] = player_impact['impact_score'] - (player_impact['traditional_rating'] / player_impact['traditional_rating'].max() * player_impact['impact_score'].max())

# Sort by rating difference
undervalued = player_impact.nlargest(10, 'rating_difference')
overvalued = player_impact.nsmallest(10, 'rating_difference')

print("Players Most Undervalued by Traditional Metrics:")
print(undervalued[['player_name', 'impact_score', 'traditional_rating', 'rating_difference', 'system_stability', 'adaptability_score', 'eigenvector_centrality']])

print("\nPlayers Most Overvalued by Traditional Metrics:")
print(overvalued[['player_name', 'impact_score', 'traditional_rating', 'rating_difference', 'system_stability', 'adaptability_score', 'eigenvector_centrality']])

In [ ]:
# Visualize traditional vs. dynamics-based ratings
plt.figure(figsize=(12, 8))
plt.scatter(player_impact['traditional_rating'], player_impact['impact_score'], alpha=0.7)

# Add diagonal line
max_trad = player_impact['traditional_rating'].max()
max_impact = player_impact['impact_score'].max()
plt.plot([0, max_trad], [0, max_impact], 'r--', alpha=0.5)

# Add labels for notable players
for i, row in undervalued.head(5).iterrows():
    plt.annotate(row['player_name'], 
                 (row['traditional_rating'], row['impact_score']),
                 fontsize=9, color='green')
    
for i, row in overvalued.head(5).iterrows():
    plt.annotate(row['player_name'], 
                 (row['traditional_rating'], row['impact_score']),
                 fontsize=9, color='red')

plt.xlabel('Traditional Rating', fontsize=12)
plt.ylabel('Dynamics-Based Impact Score', fontsize=12)
plt.title('Traditional vs. Dynamics-Based Player Evaluation', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Market Inefficiency Opportunities

Our comparison of traditional and dynamics-based player evaluation reveals several market inefficiency opportunities:

1. **Undervalued Players**: Players who contribute value through stability, adaptability, and network effects that are not captured by traditional statistics.
   - These players often have modest scoring numbers but excel in other dimensions
   - Teams could acquire these players at a discount relative to their true value
   - Examples: [List examples from your undervalued players]

2. **Overvalued Players**: Players who have impressive traditional statistics but lack stability, adaptability, or positive network effects.
   - These players may put up good numbers but contribute less to team success than their statistics suggest
   - Teams should be cautious about investing heavily in these players
   - Examples: [List examples from your overvalued players]

3. **Value Dimensions**: The most undervalued players tend to excel in stability, adaptability, and network influence, while the most overvalued players tend to be high scorers with poor stability or network metrics.

These insights can help teams identify players who are likely to outperform or underperform their traditional statistical profiles, creating opportunities for strategic player acquisition.

## Business Impact Simulation

Let's simulate the potential business impact of using our dynamics approach for team improvement.

In [ ]:
# Simulate team improvement through player acquisition
# Select a sample team
import random
sample_team_id = random.choice(games_processed['Team_ID'].unique())
sample_team_name = games_processed[games_processed['Team_ID'] == sample_team_id]['TeamName'].iloc[0]

# Get current team players
team_players = player_temporal_df[player_temporal_df['Team_ID'] == sample_team_id]['Player_ID'].unique()
team_player_names = [player_impact[player_impact['player_id'] == player_id]['player_name'].iloc[0] if player_id in player_impact['player_id'].values else str(player_id) for player_id in team_players]

print(f"Sample Team: {sample_team_name}")
print(f"Current Players: {', '.join(team_player_names)}")

# Calculate current team impact
current_impact = 0
for player_id in team_players:
    player_row = player_impact[player_impact['player_id'] == player_id]
    if len(player_row) > 0:
        current_impact += player_row['impact_score'].iloc[0]

print(f"Current Team Impact: {current_impact:.2f}")

In [ ]:
# Identify potential acquisitions
potential_acquisitions = []

for _, player in undervalued.iterrows():
    player_id = player['player_id']
    
    # Skip players already on the team
    if player_id in team_players:
        continue
    
    # Calculate synergy with current team
    team_synergy = 0
    for team_player_id in team_players:
        # Check if there's a synergy pair
        synergy_row = synergy_pairs[
            ((synergy_pairs['player1_id'] == player_id) & (synergy_pairs['player2_id'] == team_player_id)) |
            ((synergy_pairs['player1_id'] == team_player_id) & (synergy_pairs['player2_id'] == player_id))
        ]
        
        if len(synergy_row) > 0:
            team_synergy += synergy_row['synergy_score'].iloc[0]
    
    # Add to potential acquisitions
    potential_acquisitions.append({
        'player_id': player_id,
        'player_name': player['player_name'],
        'impact_score': player['impact_score'],
        'traditional_rating': player['traditional_rating'],
        'team_synergy': team_synergy,
        'total_value': player['impact_score'] + team_synergy
    })

# Convert to dataframe
acquisition_df = pd.DataFrame(potential_acquisitions)
acquisition_df = acquisition_df.sort_values('total_value', ascending=False)

# Display top acquisition targets
print(f"\nTop Acquisition Targets for {sample_team_name}:")
print(acquisition_df.head(5))

In [ ]:
# Simulate team improvement with optimal acquisitions
# Assume we can acquire the top 2 targets
top_acquisitions = acquisition_df.head(2)
acquisition_impact = top_acquisitions['total_value'].sum()

# Calculate new team impact
new_impact = current_impact + acquisition_impact
improvement = (new_impact / current_impact - 1) * 100

print(f"Current Team Impact: {current_impact:.2f}")
print(f"New Team Impact: {new_impact:.2f}")
print(f"Improvement: {improvement:.1f}%")

# Estimate win improvement
# Assume a linear relationship between impact and wins
# For simplicity, assume 1% impact improvement = 0.5 additional wins
win_improvement = improvement * 0.5 / 100 * 82  # 82 games in a season

print(f"Estimated Win Improvement: {win_improvement:.1f} additional wins")

# Estimate financial impact
# Assume each win is worth $1.5M in revenue
financial_impact = win_improvement * 1.5

print(f"Estimated Financial Impact: ${financial_impact:.1f}M in additional revenue")

### Business Impact Insights

Our business impact simulation demonstrates the potential value of using our dynamics approach for team improvement:

1. **Acquisition Targets**: The top acquisition targets for our sample team are players who not only have high impact scores but also strong synergy with existing team members.

2. **Performance Improvement**: By acquiring these players, the team could improve their overall impact by approximately [X]%, which translates to an estimated [Y] additional wins per season.

3. **Financial Impact**: The additional wins could generate approximately $[Z]M in additional revenue, representing a significant return on investment for player acquisition.

These insights demonstrate the practical business value of our dynamics approach to player evaluation and team construction. By identifying undervalued players with strong synergy potential, teams can make more efficient use of their resources and achieve better results on the court.

## Applications & Future Work

Let's explore potential applications of our dynamics approach and directions for future research.

### Game Strategy Recommendations

Our dynamics approach can inform game strategy in several ways:

1. **Lineup Optimization**: Use stability profiles and synergy pairs to construct lineups that maximize performance.
   - Start with high-value stability players as the core
   - Add high-ceiling volatility players strategically
   - Ensure lineup includes players with strong synergy

2. **Matchup Exploitation**: Target opponent weaknesses based on stability patterns.
   - Against teams with volatile players, focus on defensive consistency to limit their ceiling
   - Against teams with stable players, use high-ceiling volatility players to create variance

3. **In-Game Adjustments**: Use stability profiles to inform substitution patterns.
   - When leading, rely on high-value stability players to maintain the lead
   - When trailing, insert high-ceiling volatility players to change the game dynamic

4. **End-Game Situations**: Use stability profiles to select players for critical moments.
   - In close games, rely on players with high stability and positive impact
   - In blowouts, give playing time to players who need development

### Player Acquisition Strategy

Our dynamics approach can also inform player acquisition strategy:

1. **Target Undervalued Players**: Focus on players who are undervalued by traditional metrics but score highly on our dynamics-based impact score.
   - Look for players with high stability, adaptability, and positive network effects
   - Consider team fit and potential synergies with current roster

2. **Avoid Overvalued Players**: Be cautious about players who score highly on traditional metrics but poorly on our dynamics-based impact score.
   - Watch out for players with high volatility, low adaptability, or negative network effects
   - Consider whether their playing style fits with the team's system

3. **Balance Roster Construction**: Maintain a balance of stability and volatility in the roster.
   - Build around a core of high-value stability players
   - Add high-ceiling volatility players for specific roles
   - Avoid accumulating too many high-risk variability players

4. **Consider Network Effects**: Prioritize players who have positive network effects and can make teammates better.
   - Look for players with high eigenvector centrality and positive influence
   - Consider existing synergy pairs when making acquisition decisions

### Future Research Directions

There are several promising directions for future research:

1. **Spatial Analysis**: Incorporate spatial data to analyze how player performance varies by court region.
   - Map stability and volatility patterns across different areas of the court
   - Identify players who excel in specific spatial contexts

2. **Temporal Evolution**: Track stability metrics over time to identify career transition points.
   - Analyze how stability profiles evolve throughout a player's career
   - Identify early indicators of performance decline or improvement

3. **Advanced Network Models**: Develop more sophisticated network models to capture complex teammate interactions.
   - Incorporate directed influence relationships
   - Model higher-order interactions beyond pairwise relationships

4. **Predictive Modeling**: Develop predictive models for player performance based on stability profiles and network position.
   - Forecast how players will perform in new team contexts
   - Predict career trajectories based on stability patterns

5. **Integration with Tracking Data**: Combine our dynamics approach with player tracking data for deeper insights.
   - Analyze how movement patterns relate to stability profiles
   - Identify physical indicators of performance consistency

## Save Player Impact Results

Let's save our player impact results for future reference.

In [ ]:
# Save player impact data
player_impact.to_csv('../data/processed/player_impact.csv', index=False)
print(f"Saved player impact data to ../data/processed/player_impact.csv")

## Conclusion

In this notebook, we've developed a comprehensive player impact framework that integrates production, stability, adaptability, and network metrics. We've compared traditional and dynamics-based player evaluation, simulated the business impact of our approach, and explored potential applications and future research directions.

Key accomplishments:
1. Integrated multiple dimensions of player performance into a comprehensive impact framework
2. Identified players who are undervalued or overvalued by traditional metrics
3. Simulated the potential business impact of using our dynamics approach for team improvement
4. Developed game strategy and player acquisition recommendations based on our findings
5. Outlined promising directions for future research

Our multi-dimensional approach to player evaluation provides a more nuanced understanding of player value than traditional statistics alone, capturing aspects of performance that are invisible to conventional metrics. By considering stability, adaptability, and network effects alongside traditional production metrics, teams can make more informed decisions about player acquisition, lineup construction, and game strategy, potentially gaining a competitive advantage in the NBA marketplace.